In [1]:
# !python3 -m pip uninstall tensorflow tensorflow-probability nsc -y
# !python3 -m pip install tensorflow tensorflow-probability -q

In [2]:
!python3 -m pip uninstall nsc -y -q
!python3 -m pip install -i https://test.pypi.org/simple/ nsc -q

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set()
from collections import defaultdict
import math
from typing import List

# From nsc lib
from nsc.util import function as nsc_func
# from nsc.util.function import coupled_logarithm, coupled_exponential

Importing NSC lib v0.0.1.6.


### 1) CoupledNormalDistribution function version

In [4]:
def norm_CG(sigma, kappa):
    if kappa == 0:
        result = math.sqrt(2*math.pi) * sigma
    elif kappa < 0:
        result = math.sqrt(math.pi) * sigma * math.gamma((-1+kappa) / (2*kappa)) / float(math.sqrt(-1*kappa) * math.gamma(1 - (1 / (2*kappa))))
    else:
        result = math.sqrt(math.pi) * sigma * math.gamma(1 / (2*kappa)) / float(math.sqrt(kappa) * math.gamma((1+kappa)/(2*kappa)))
  
    return result

In [5]:
def CoupledNormalDistribution(mean, sigma, kappa, alpha):
    """
    Short description
    
    Inputs
    ----------
    x : Input variable in which the coupled logarithm is applied to.
    mean : 
    sigma : 
    kappa : Coupling parameter which modifies the coupled logarithm function.
    dim : The dimension of x, or rank if x is a tensor. Not needed?
    """

    assert sigma > 0, "std must be greater than 0."
    assert alpha in [1, 2], "alpha must be set to either 1 or 2."

    
    if kappa >= 0:
        input = np.arange(mean-20, mean+20, (20+mean - -20+mean)/(2**16+1))
    else:
        x1 = mean - ((-1*sigma**2) / kappa)**0.5
        x2 = mean + ((-1*sigma**2) / kappa)**0.5
        input = np.arange(mean - ((-1*sigma**2) / kappa)**0.5, mean + ((-1*sigma**2) / kappa)**0.5, (x2-x1)/(2**16+1))
 
    normCGvalue = 1/float(norm_CG(sigma, kappa))
    
    coupledNormalDistributionResult = normCGvalue * (nsc_func.coupled_exponential((input - mean)**2/sigma**2, kappa)) ** -0.5
  
    return coupledNormalDistributionResult

In [6]:
kappa, alpha, dim = 0.5, 2, 1

In [7]:
mu, sigma = 0, 1 # mean and standard deviation
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = CoupledNormalDistribution(mu, sigma, kappa, alpha)

dx = np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[1] - np.arange(mu-20, mu+20, (20+mu - -20+mu)/(2**16+1))[0]

### 2) CoupledNormalDistribution class

In [8]:
class CoupledNormal:
    """Coupled Normal Distribution.

    This distribution has parameters: location `loc`, 'scale', coupling `kappa`,
    and `alpha`.

    """
    def __init__(self,
                 loc: [int, float, List, np.ndarray],
                 scale: [int, float, List, np.ndarray],
                 kappa: [int, float] = 0.,
                 alpha: int = 2,
                 validate_args: bool = False,
                 allow_nan_stats: bool = True
                 ):
        loc = np.asarray(loc) if isinstance(loc, List) else loc
        scale = np.asarray(scale) if isinstance(scale, List) else scale
        if validate_args:
            assert isinstance(loc, (int, float, np.ndarray)), "loc must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert isinstance(scale, (int, float, np.ndarray)), "scale must be either an int/float type for scalar, or an list/ndarray type for multidimensional."
            assert type(loc) == type(scale), "loc and scale must be the same type."
            if instance(loc, np.ndarray):
                assert loc.shape == scale.shape, "loc and scale must have the same dimensions (check respective .shape())."
                assert np.all((scale >= 0)), "All scale values must be greater or equal to 0."            
            else:
                assert scale >= 0, "scale must be greater or equal to 0."            
            assert isinstance(kappa, (int, float)), "kappa must be an int or float type."
            assert isinstance(alpha, int), "alpha must be an int that equals to either 1 or 2."
            assert alpha in [1, 2], "alpha must be equal to either 1 or 2."
        self.loc = loc
        self.scale = scale
        self.kappa = kappa
        self.alpha = alpha
        print(f"<nsc.distributions.CoupledNormal batch_shape={self._batch_shape()} event_shape={self._event_shape()}>")

    def n_dim(self):
        return 1 if self._event_shape() == [] else self._event_shape()[0]
    
    def _batch_shape(self) -> List:
        if isinstance(self.loc, (int, float)):
            # return [] signifying single batch of a single distribution
            return []
        elif self.loc.shape[0] == 1:
            # return [] signifying single batch of a multivariate distribution
            return []
        else:
            # return [batch size]
            return [self.loc.shape[0]]

    def _event_shape(self) -> List:
        if isinstance(self.loc, (int, float)):
            # return [] signifying single random variable
            return []
        elif len(self.loc.shape) == 1:
            # return [] signifying single random variable (although with multiple batches)
            return []
        else:
            # return [n of random variables]
            return [self.loc.shape[-1]]

    def prob(self, X: [List, np.ndarray]):
        # Check whether input X is valid
        X = np.asarray(X) if isinstance(X, List) else X
        assert isinstance(X, np.ndarray), "X must be a List or np.ndarray."
        # assert type(X[0]) == type(self.loc), "X samples must be the same type as loc and scale."
        if isinstance(X[0], np.ndarray):
            assert X[0].shape == loc.shape, "X samples must have the same dimensions as loc and scale (check respective .shape())."
        # Calculate PDF with input X
        X_norm = (X-self.loc)**2 / self.scale**2
        norm_term = self._normalized_term()
        p = (nsc_func.coupled_exponential(X_norm, kappa))**-0.5 / norm_term
        # normCGvalue =  1/float(norm_CG(sigma, kappa))
        # coupledNormalDistributionResult = normCGvalue * (fj.coupled_exponential(y, kappa)) ** -0.5
        return p
    
    # Normalization of 1-D Coupled Gaussian (NormCG)
    def _normalized_term(self) -> [int, float, np.ndarray]:
        if self.kappa == 0:
            norm_term = math.sqrt(2*math.pi) * self.scale
        elif self.kappa < 0:
            gamma_num = math.gamma(self.kappa-1) / (2*self.kappa)
            gamma_dem = math.gamma(1 - (1 / (2*self.kappa)))
            norm_term = (math.sqrt(math.pi)*self.sigma*gamma_num) / float(math.sqrt(-1*self.kappa)*gamma_dem)
        else:
            gamma_num = math.gamma(1 / (2*self.kappa))
            gamma_dem = math.gamma((1+kappa)/(2*self.kappa))
            norm_term = (math.sqrt(math.pi)*self.sigma*gamma_num) / float(math.sqrt(self.kappa)*gamma_dem)
        return norm_term


***Test***

In [9]:
mu, sigma, kappa, alpha = 0., 1., 0.5, 2
X_input = np.arange(mu-20., mu+20., (20.+mu - -20.+mu)/(2.**16.+1.), dtype=float)

In [10]:
X_input

array([-20.        , -19.99938966, -19.99877932, ...,  19.99816897,
        19.99877932,  19.99938966])

Coupled Normal distribution

In [11]:
cn = CoupledNormal(loc=mu, scale=sigma)

<nsc.distributions.CoupledNormal batch_shape=[] event_shape=[]>


In [12]:
print(cn.n_dim())
print(cn._normalized_term())
print(cn.prob(X_input))

1
2.5066282746310002
[0.00014    0.00014001 0.00014002 ... 0.00014003 0.00014002 0.00014001]


Coupled Normal multiple distributions (i.e., batch size of 2)

In [13]:
cn = CoupledNormal(loc=[0., 1.], scale=[1., 2.])

<nsc.distributions.CoupledNormal batch_shape=[2] event_shape=[]>


In [14]:
print(cn.n_dim())
print(cn._normalized_term())
# print(cn.prob(X_input))

1
[2.50662827 5.01325655]


Multivariate Coupled Normal distribution

In [15]:
# use double brackets to signify multiple multivariate
cn = CoupledNormal(loc=[[0., 1.]], scale=[[1., 2.]])

<nsc.distributions.CoupledNormal batch_shape=[] event_shape=[2]>


In [16]:
print(cn.n_dim())
print(cn._normalized_term())
# print(cn.prob(X_input))

2
[[2.50662827 5.01325655]]


Multivariate Coupled Normal multiple distribution (i.e., batch size of 3)

In [17]:
cn = CoupledNormal(loc=[[0., 1.], [1., 2.], [2., 3.]], \
                   scale=[[0., 1.], [2., 3.], [4., 5.]]
                   )
cn.n_dim()

<nsc.distributions.CoupledNormal batch_shape=[3] event_shape=[2]>


2

In [18]:
print(cn.n_dim())
print(cn._normalized_term())
# print(cn.prob(X_input))

2
[[ 0.          2.50662827]
 [ 5.01325655  7.51988482]
 [10.0265131  12.53314137]]


In [19]:
cn = CoupledNormal(loc=[[0., 1., 0., 1.], [1., 2., 1., 2.], [2., 3., 2., 3.]], \
                   scale=[[0., 1., 0., 1.], [2., 3., 2., 3.], [4., 5., 4., 5.]]
                   )
cn.n_dim()

<nsc.distributions.CoupledNormal batch_shape=[3] event_shape=[4]>


4

In [20]:
print(cn.n_dim())
print(cn._normalized_term())
# print(cn.prob(X_input))

4
[[ 0.          2.50662827  0.          2.50662827]
 [ 5.01325655  7.51988482  5.01325655  7.51988482]
 [10.0265131  12.53314137 10.0265131  12.53314137]]
